In [ ]:
!unzip "/content/drive/MyDrive/dialogueText.csv"

Archive:  /content/drive/MyDrive/dialogueText.csv
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/dialogueText.csv or
        /content/drive/MyDrive/dialogueText.csv.zip, and cannot find /content/drive/MyDrive/dialogueText.csv.ZIP, period.


In [ ]:
!pip install dataset datasets
!pip install torch
!pip install pandas
!pip install tqdm


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import multiprocessing
import numpy

import pandas as pd
from tqdm import tqdm
import torch
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset, Features, Value, ClassLabel


In [ ]:
import pandas as pd
import numpy as np
import re

# Load the data
data = pd.read_csv('/content/drive/MyDrive/dialogueText.csv')

data = data[['dialogueID', 'text']]

# Select necessary columns
data = data[['dialogueID', 'text']]

# Ensure 'text' column is of string type
data['text'] = data['text'].astype(str)

# Remove missing values
data.dropna(inplace=True)


# Remove duplicate rows
data.drop_duplicates(inplace=True)

# Remove special characters and numbers from text
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text

data['text'] = data['text'].apply(clean_text)



# Convert text to lowercase
data['text'] = data['text'].str.lower()



# Remove short texts
min_length = 5
data= data[(data['text'].str.len() >= min_length) ]


# Take only 20% of the data
data = data.sample(frac=0.2, random_state=1)


# Save the cleaned data
data.to_csv('/content/drive/MyDrive/dc/cleaned_train_dialogueText.csv', index=False)



In [ ]:
data.head()

,dialogueID,text
249721,70990.tsv,lets do it
999394,51037.tsv,ive used it on my desktop for a year
247696,153991.tsv,i basically need to know if photo quality prin...
953276,275418.tsv,what is better
83082,60473.tsv,in ubuntu period


In [ ]:
data.columns

Index(['dialogueID', 'text'], dtype='object')

In [ ]:
cleaned_data =pd.read_csv("/content/drive/MyDrive/dc/cleaned_train_dialogueText.csv")

In [ ]:
print(cleaned_data)

         label                                               text
0       145240                                         lets do it
1       134855               ive used it on my desktop for a year
2        27913  i basically need to know if photo quality prin...
3        90296                                     what is better
4       139789                                   in ubuntu period
...        ...                                                ...
194479  159601                           thats no joke thats true
194480   17710                             linux please not lunix
194481  135139                                     maybe in china
194482   38670  i have gigs of ram now play a lot of cedega ga...
194483   52777                did you even look at what he pasted

[194484 rows x 2 columns]


In [ ]:
cleaned_data=cleaned_data.rename(columns={"dialogueID":"label", "text":"text"})

# If the 'label' needs to be converted to categorical data
cleaned_data['label'] = cleaned_data['label'].astype('category').cat.codes

# Define the schema with explicit features
features = Features({'text': Value('string'), 'label': ClassLabel(names=cleaned_data['label'].astype(str).unique().tolist())})


# Create the Dataset from pandas DataFrame with explicit features
dataset = Dataset.from_pandas(cleaned_data, features=features)

# Split the dataset into train and test sets
train_test_split = dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Print a few samples to verify
print(train_dataset[0])
print(train_dataset[1])

# Save the preprocessed dataset for future use
train_dataset.save_to_disk('/content/drive/MyDrive/dc/train_dataset_dialogueText')
eval_dataset.save_to_disk('/content/drive/MyDrive/dc/eval_dataset_dialogueText')


{'text': 'perl and python which is better', 'label': 35700}
{'text': 'actually mine says battary not present but thats because my battery is lucky to last minutes', 'label': 157040}


Saving the dataset (0/1 shards):   0%|          | 0/175035 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/19449 [00:00<?, ? examples/s]